In [ ]:
import os
import re
from skimage.filters import threshold_otsu,gaussian
from skimage import measure
import pandas as pd
import numpy as nm
from sklearn.preprocessing import MinMaxScaler
from skimage.io import imread
from skimage import color
from skimage.transform import resize
from numpy import asarray
import matplotlib.pyplot as plt


In [ ]:
#extract_only signal from images
def extract_signal_leads(Leads,folder_name,parent):
  #looping through image list containg all leads from 1-13
  for x,y in enumerate(Leads):
    #creating subplot
    fig1 , ax1 = plt.subplots()

    #set fig size
    #fig1.set_size_inches(20, 20)

    #converting to gray scale
    grayscale = color.rgb2gray(y)
    #smoothing image
    blurred_image = gaussian(grayscale,sigma=0.7)
    #thresholding to distinguish foreground and background
    #using otsu thresholding for getting threshold value
    global_thresh = threshold_otsu(blurred_image)

    #creating binary image based on threshold
    binary_global = blurred_image < global_thresh

    #resize image
    if x!=12:
      binary_global = resize(binary_global, (300, 450))

    ax1.imshow(binary_global,cmap="gray")
    ax1.axis('off')
    ax1.set_title("pre-processed Leads {} image".format(x+1))
    plt.close('all')
    plt.ioff()
    #save the image
    fig1.savefig('{parent}/{folder_name}/Lead_{x}_preprocessed_Signal.png'.format(folder_name=folder_name,x=x+1,parent=parent))

    fig7 , ax7 = plt.subplots()
    plt.gca().invert_yaxis()

    #find contour and get only the necessary signal contour
    contours = measure.find_contours(binary_global,0.8)
    contours_shape = sorted([x.shape for x in contours])[::-1][0:1]
    for contour in contours:
      if contour.shape in contours_shape:
        test = resize(contour, (200, 2))
        ax7.plot(test[:, 1], test[:, 0],linewidth=1,color='black')
    ax7.axis('image')
    ax7.set_title("Contour {} image".format(x+1))
    plt.close('all')
    plt.ioff()
    #save the image
    fig7.savefig('{parent}/{folder_name}/Lead_{x}_Contour_Signal.png'.format(folder_name=folder_name,x=x+1,parent=parent))
    lead_no=x
    #convert_csv(test,lead_no,folder_name,parent)
    #scale_csv(test,lead_no,folder_name,parent)
    scale_csv_1D(test,lead_no,folder_name,parent)

In [ ]:
"""##FUNCTION FOR  IMAGE LEADS(1-13) PRE-PROCESSING**"""
def Convert_Image_Lead(image_file,parent_folder):
    #read the image
    image=imread('{parent}/{image_file}'.format(parent=str(parent_folder),image_file=str(image_file)),plugin='matplotlib')
    #dividing the ECG leads from 1-13 from the above image
    Lead_1 = image[300:600, 150:643]
    Lead_2 = image[300:600, 646:1135]
    Lead_3 = image[300:600, 1140:1626]
    Lead_4 = image[300:600, 1630:2125]
    Lead_5 = image[600:900, 150:643]
    Lead_6 = image[600:900, 646:1135]
    Lead_7 = image[600:900, 1140:1626]
    Lead_8 = image[600:900, 1630:2125]
    Lead_9 = image[900:1200, 150:643]
    Lead_10 = image[900:1200, 646:1135]
    Lead_11 = image[900:1200, 1140:1626]
    Lead_12 = image[900:1200, 1630:2125]
    Lead_13 = image[1250:1480, 150:2125]

    #list of leads
    Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12,Lead_13]

    #folder_name to store lead_images
    folder_name= re.sub('.jpg', '',image_file)

    #loop through leads and create seperate images
    for x,y in enumerate(Leads):
      fig , ax = plt.subplots()
      #fig.set_size_inches(20, 20)
      ax.imshow(y)
      ax.axis('off')
      ax.set_title("Leads {0}".format(x+1))
      if (os.path.exists(parent_folder+'/'+folder_name)):
        pass
      else:
        os.makedirs(parent_folder+'/'+folder_name)

      #save the image
      plt.close('all')
      plt.ioff()
      fig.savefig('{parent}/{folder_name}/Lead_{x}_Signal.png'.format(folder_name=folder_name,x=x+1,parent=parent_folder))

    extract_signal_leads(Leads,folder_name,parent_folder)

In [ ]:
def convert_csv(test,lead_no,folder_name,parent):
#convert contour to dataframe
  target=folder_name[0:2]
  df = pd.DataFrame(test, columns = ['X','Y'])
  df['Target']=target
  #x_axis= 'Lead_{lead_no}_X'.format(lead_no=lead_no)
  #y_axis= 'Lead_{lead_no}_Y'.format(lead_no=lead_no)
  fig5, ax5 = plt.subplots()
  #convert to CSV
  df.to_csv('{parent}/{folder_name}/{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent,folder_name=folder_name),index=False)

def scale_csv(test,lead_no,folder_name,parent):
  #scaling the data and testing
  target=folder_name[0:2]
  scaler = MinMaxScaler()
  fit_transform_data = scaler.fit_transform(test)
  Normalized_Scaled=pd.DataFrame(fit_transform_data, columns = ['X','Y'])
  Normalized_Scaled=Normalized_Scaled.T
  Normalized_Scaled['Target']=target
  #scaled_data to CSV
  if (os.path.isfile('{parent}/Scaled_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent))):
    Normalized_Scaled.to_csv('{parent}/Scaled_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent), mode='a', header=False,index=False)
  else:
    Normalized_Scaled.to_csv('{parent}/Scaled_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent,folder_name=folder_name),index=False)

In [ ]:
def scale_csv_1D(test,lead_no,folder_name,parent):
  target=folder_name[0:2]
  #scaling the data and testing
  scaler = MinMaxScaler()
  fit_transform_data = scaler.fit_transform(test)
  Normalized_Scaled=pd.DataFrame(fit_transform_data[:,0], columns = ['X'])
  fig6, ax6 = plt.subplots()
  plt.gca().invert_yaxis()
  ax6.plot(Normalized_Scaled,linewidth=1,color='black',linestyle='solid')
  plt.close('all')
  plt.ioff()
  fig6.savefig('{parent}/{folder_name}/ID_Lead_{lead_no}_Signal.png'.format(folder_name=folder_name,lead_no=lead_no+1,parent=parent))
  Normalized_Scaled=Normalized_Scaled.T
  Normalized_Scaled['Target']=target
  #scaled_data to CSV
  if (os.path.isfile('{parent}/scaled_data_1D_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent))):
    Normalized_Scaled.to_csv('{parent}/scaled_data_1D_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent), mode='a', header=False,index=False)
  else:
    Normalized_Scaled.to_csv('{parent}/scaled_data_1D_{lead_no}.csv'.format(lead_no=lead_no+1,parent=parent,folder_name=folder_name),index=False)

In [ ]:
#load the different types of folders
normal_parent_dir = '/content/drive/MyDrive/Arjun_Workspace/Final_2/Normal'
abnormal_parent_dir = '/content/drive/MyDrive/Arjun_Workspace/Final_2/Abnormal HB'
MI_parent_dir = '/content/drive/MyDrive/Arjun_Workspace/Final_2/MI'
MI_history_parent_dir = '/content/drive/MyDrive/Arjun_Workspace/Final_2/HMI'

#Types of heart.
Types_ECG = {'Abnormal_heart_beat':abnormal_parent_dir,'MI':MI_parent_dir,'History_MI':MI_history_parent_dir,'Normal':normal_parent_dir}

#Types_ECG_test = {'Abnormal':abnormal_parent_dir}

#### parse all the files in the different folders and peform all the above steps as mentioned in the analysis Sample.

#loop through folder/files and create seperate images of different leads
for types,folder in Types_ECG.items():
  for files in os.listdir(folder):
    #if files.endswith(".jpg"):
    with open(os.path.join(folder, files), 'r') as f:
      Convert_Image_Lead(files,folder)

IsADirectoryError: ignored

In [ ]:
import pandas as pd
import numpy as np
import os
from natsort import natsorted
import joblib

In [ ]:
#creating list to store file_names
NORMAL_=[]
MI_=[]
PMI_=[]
HB_=[]

normal = '/content/drive/MyDrive/Arjun_Workspace/Final_2/Normal/normal_scaled'
abnormal = '/content/drive/MyDrive/Arjun_Workspace/Final_2/Abnormal HB/abnormal_scaled'
MI = '/content/drive/MyDrive/Arjun_Workspace/Final_2/MI/MI_scaled'
MI_history = '/content/drive/MyDrive/Arjun_Workspace/Final_2/HMI/history_scaled'

Types_ECG = {'normal':normal,'Abnormal_heart_beat':abnormal,'MI':MI,'History_MI':MI_history}

for types,folder in Types_ECG.items():
  for files in os.listdir(folder):
    if types=='normal':
      NORMAL_.append(files)
    elif types=='Abnormal_heart_beat':
      HB_.append(files)
    elif types=='MI':
      MI_.append(files)
    elif types=='History_MI':
      PMI_.append(files)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
NORMAL_ = natsorted(NORMAL_)
NORMAL_

['scaled_data_1D_1.csv',
 'scaled_data_1D_2.csv',
 'scaled_data_1D_3.csv',
 'scaled_data_1D_4.csv',
 'scaled_data_1D_5.csv',
 'scaled_data_1D_6.csv',
 'scaled_data_1D_7.csv',
 'scaled_data_1D_8.csv',
 'scaled_data_1D_9.csv',
 'scaled_data_1D_10.csv',
 'scaled_data_1D_11.csv',
 'scaled_data_1D_12.csv',
 'scaled_data_1D_13.csv']

In [ ]:
MI_ = natsorted(MI_)
MI_

['scaled_data_1D_1.csv',
 'scaled_data_1D_2.csv',
 'scaled_data_1D_3.csv',
 'scaled_data_1D_4.csv',
 'scaled_data_1D_5.csv',
 'scaled_data_1D_6.csv',
 'scaled_data_1D_7.csv',
 'scaled_data_1D_8.csv',
 'scaled_data_1D_9.csv',
 'scaled_data_1D_10.csv',
 'scaled_data_1D_11.csv',
 'scaled_data_1D_12.csv',
 'scaled_data_1D_13.csv']

In [ ]:
PMI_ = natsorted(PMI_)
PMI_

['scaled_data_1D_1.csv',
 'scaled_data_1D_2.csv',
 'scaled_data_1D_3.csv',
 'scaled_data_1D_4.csv',
 'scaled_data_1D_5.csv',
 'scaled_data_1D_6.csv',
 'scaled_data_1D_7.csv',
 'scaled_data_1D_8.csv',
 'scaled_data_1D_9.csv',
 'scaled_data_1D_10.csv',
 'scaled_data_1D_11.csv',
 'scaled_data_1D_12.csv',
 'scaled_data_1D_13.csv']

In [ ]:
HB_ = natsorted(HB_)
HB_

['scaled_data_1D_1.csv',
 'scaled_data_1D_2.csv',
 'scaled_data_1D_3.csv',
 'scaled_data_1D_4.csv',
 'scaled_data_1D_5.csv',
 'scaled_data_1D_6.csv',
 'scaled_data_1D_7.csv',
 'scaled_data_1D_8.csv',
 'scaled_data_1D_9.csv',
 'scaled_data_1D_10.csv',
 'scaled_data_1D_11.csv',
 'scaled_data_1D_12.csv',
 'scaled_data_1D_13.csv']

In [ ]:
#loop over and create combined csv files for each leads.
for x in range(len(MI_)):
  df1=pd.read_csv('/content/drive/MyDrive/Arjun_Workspace/Final_2/Normal/normal_scaled/{}'.format(NORMAL_[x]))
  df2=pd.read_csv('/content/drive/MyDrive/Arjun_Workspace/Final_2/Abnormal HB/abnormal_scaled/{}'.format(HB_[x]))
  df3=pd.read_csv('/content/drive/MyDrive/Arjun_Workspace/Final_2/MI/MI_scaled/{}'.format(MI_[x]))
  df4=pd.read_csv('/content/drive/MyDrive/Arjun_Workspace/Final_2/HMI/history_scaled/{}'.format(PMI_[x]))
  final_df = pd.concat([df1,df2,df3,df4],ignore_index=True)
  final_df.to_csv('/content/drive/MyDrive/Arjun_Workspace/Final_2/Combined/Combined_IDLead_{}.csv'.format(x+1))

NameError: ignored

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Arjun_Workspace/Final_1/Combined/Combined_IDLead_1.csv')
df['Target'].unique()

FileNotFoundError: ignored

In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
encode_target_label = df.groupby('Target').ngroup().rename("target").to_frame()
test_final  = df.merge(encode_target_label, left_index=True, right_index=True)
test_final.drop(columns=['Target'],inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,target
0,0.888626,0.812245,0.665561,0.500157,0.390310,0.503157,0.671016,0.831373,0.930915,0.885081,...,0.403099,0.227493,0.071006,0.036642,0.182108,0.354795,0.526547,0.687533,0.803033,2
1,0.807023,0.844462,0.858971,0.862217,0.849314,0.837517,0.832122,0.804881,0.745785,0.703045,...,0.706297,0.765693,0.784813,0.795663,0.809886,0.823817,0.811508,0.799580,0.752555,2
2,0.653819,0.709960,0.754838,0.817140,0.915838,1.000000,0.955020,0.839406,0.715041,0.594235,...,0.527485,0.653147,0.777326,0.861060,0.801774,0.726080,0.689121,0.644085,0.580488,2
3,0.936459,0.936513,0.943110,0.941819,0.936485,0.936999,0.947314,0.923002,0.844092,0.757641,...,0.706292,0.800846,0.852939,0.859978,0.859977,0.854643,0.853353,0.859949,0.860003,2
4,0.633524,0.711548,0.790842,0.857253,0.911213,0.939912,0.934426,0.894882,0.812718,0.706587,...,0.747938,0.842870,0.898640,0.907277,0.872172,0.810854,0.739755,0.659590,0.586202,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,0.899701,0.925065,0.953176,0.970737,0.972880,0.987581,0.999396,0.947425,0.817473,0.682324,...,0.715492,0.843744,0.931497,0.928955,0.914313,0.918867,0.894579,0.864759,0.846643,3
488,0.882043,0.903099,0.954086,1.000000,0.982167,0.943872,0.910847,0.907839,0.912979,0.901471,...,0.859483,0.852511,0.853891,0.883268,0.926698,0.945426,0.897190,0.851191,0.831221,3
489,0.792318,0.805000,0.787662,0.767232,0.802106,0.861793,0.868489,0.819355,0.766635,0.748084,...,0.726346,0.774266,0.825273,0.802655,0.740945,0.723790,0.751791,0.762488,0.749903,3
490,0.753196,0.742908,0.727618,0.711634,0.693631,0.685538,0.696426,0.683035,0.657818,0.666619,...,0.607504,0.630874,0.627155,0.624694,0.633411,0.650912,0.673251,0.684370,0.686523,3


In [ ]:
#just for testing
# Now Perform Dimensionality reduction (PCA) on that Dataframe and check
from sklearn.decomposition import PCA

#do PCA and choose componeents as 15
pca = PCA(n_components=15)
x_pca = pca.fit_transform(test_final.iloc[:,0:-1])
x_pca = pd.DataFrame(x_pca)

# Calculate the variance explained by priciple components
explained_variance = pca.explained_variance_ratio_
print('Variance of each component:', pca.explained_variance_ratio_)
print('\n Total Variance Explained:', round(sum(list(pca.explained_variance_ratio_))*100, 2))

#store the new pca generated dimensions in a dataframe
pca_df = pd.DataFrame(data = x_pca)
target = pd.Series(test_final['target'], name='target')
result_df = pd.concat([pca_df, target], axis=1)
result_df

NameError: ignored

In [ ]:
result_df

,0,1,2,3,4,5,6,7,8,9,target
0,-0.549375,0.494565,-0.045338,1.422135,-0.268344,-1.590527,0.449342,0.875116,0.537622,-0.007312,2
1,-0.863060,-1.580209,0.514307,0.182243,-0.451672,-0.761466,-0.419687,-0.979922,-0.977254,-0.150150,2
2,-0.278309,0.626545,0.690094,-0.884929,-0.431733,1.087767,0.898435,0.482051,-0.000227,0.721407,2
3,-1.836973,-0.446914,1.372553,-0.256165,0.608453,0.087004,0.427743,-0.113751,0.104299,0.210230,2
4,-1.132701,-0.717709,0.090382,-1.262666,0.533160,0.439906,0.885697,0.572187,0.100195,-1.086943,2
...,...,...,...,...,...,...,...,...,...,...,...
487,-2.157170,-0.427223,0.292471,-1.614653,0.336949,-0.001895,0.782310,1.043056,-0.243457,-0.845600,3
488,-1.523648,-1.585517,0.471800,-1.570187,0.351871,0.258667,-0.013342,0.827453,0.109311,-0.819523,3
489,-0.200692,-0.293620,-1.004880,0.506783,1.626634,-0.473461,0.447730,0.041822,0.146146,0.037439,3
490,0.080288,0.438342,1.051315,0.642258,0.392410,-0.196922,0.539831,-0.720026,-0.312818,0.804052,3


In [ ]:
  # Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('knn', KNeighborsClassifier())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)


k_range = list(range(1, 9))
parameters = dict(knn__n_neighbors=k_range)

#input
X = result_df.iloc[:,0:-1]

#target
y=result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=42)


cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

Knn_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
print("Accuracy: {}".format(Knn_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.4268292682926829
              precision    recall  f1-score   support

           0       0.22      0.34      0.27        29
           1       0.17      0.14      0.15        28
           2       0.53      0.59      0.56       106
           3       0.48      0.34      0.40        83

    accuracy                           0.43       246
   macro avg       0.35      0.35      0.35       246
weighted avg       0.44      0.43      0.42       246

Tuned Model Parameters: {'knn__n_neighbors': 5}


LOGISTIC REGRESSION

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('lr', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

#input
X = result_df.iloc[:,0:-1]

#target
y=result_df.iloc[:,-1]


c_space = np.logspace(-4, 4, 3)
parameters = {'lr__C': c_space,'lr__penalty': ['l2']}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#call GridSearchCV and set crossvalscore to 2
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)
LR_Accuracy = cv.score(X_test, y_test)

In [ ]:
# Compute and print metrics
print("Accuracy: {}".format(LR_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.4416243654822335
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        24
           1       0.00      0.00      0.00        20
           2       0.44      1.00      0.61        87
           3       0.00      0.00      0.00        66

    accuracy                           0.44       197
   macro avg       0.11      0.25      0.15       197
weighted avg       0.20      0.44      0.27       197

Tuned Model Parameters: {'lr__C': 0.0001, 'lr__penalty': 'l2'}


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM

In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline
steps = [('SVM', SVC())]

pipeline = Pipeline(steps)

#input
X = result_df.iloc[:,0:-1]

#target
y=result_df.iloc[:,-1]


parameters = {'SVM__C':[10],'SVM__gamma':[1]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=21)

cv = GridSearchCV(pipeline,parameters,cv=3)
cv.fit(X_train,y_train)

y_pred = cv.predict(X_test)
SVM_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
SVM_Accuracy=cv.score(X_test, y_test)

print("Accuracy: {}".format(SVM_Accuracy))
print(classification_report(y_test, y_pred))

Accuracy: 0.4619289340101523
              precision    recall  f1-score   support

           0       0.56      0.24      0.33        21
           1       0.88      0.23      0.37        30
           2       0.50      0.61      0.55        90
           3       0.35      0.43      0.38        56

    accuracy                           0.46       197
   macro avg       0.57      0.38      0.41       197
weighted avg       0.52      0.46      0.45       197



NOW COMBINING ALL 12 LEADS INTO A SINGLE CSV FILE AND THEN PERFROM MODEL ANALYSIS

In [ ]:
#lets try combining all 12 leads in a single csv
location= '/content/drive/MyDrive/Arjun_Workspace/Final_2'
for files in natsorted(os.listdir(location)):
  if files.endswith(".csv") and not files.endswith("13.csv"):
    if files!='Combined_IDLead_1.csv':
      df=pd.read_csv('/content/drive/MyDrive/Arjun_Workspace/Final_2/{}'.format(files))
      df.drop(columns=['Unnamed: 0'],inplace=True)
      test_final=pd.concat([test_final,df],axis=1,ignore_index=True)
      test_final.drop(columns=test_final.columns[-1],axis=1,inplace=True)

#drop the target column
#test_final.drop(columns=[200],axis=1,inplace=True)
test_final

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,target
0,0.888626,0.812245,0.665561,0.500157,0.390310,0.503157,0.671016,0.831373,0.930915,0.885081,...,0.403099,0.227493,0.071006,0.036642,0.182108,0.354795,0.526547,0.687533,0.803033,2
1,0.807023,0.844462,0.858971,0.862217,0.849314,0.837517,0.832122,0.804881,0.745785,0.703045,...,0.706297,0.765693,0.784813,0.795663,0.809886,0.823817,0.811508,0.799580,0.752555,2
2,0.653819,0.709960,0.754838,0.817140,0.915838,1.000000,0.955020,0.839406,0.715041,0.594235,...,0.527485,0.653147,0.777326,0.861060,0.801774,0.726080,0.689121,0.644085,0.580488,2
3,0.936459,0.936513,0.943110,0.941819,0.936485,0.936999,0.947314,0.923002,0.844092,0.757641,...,0.706292,0.800846,0.852939,0.859978,0.859977,0.854643,0.853353,0.859949,0.860003,2
4,0.633524,0.711548,0.790842,0.857253,0.911213,0.939912,0.934426,0.894882,0.812718,0.706587,...,0.747938,0.842870,0.898640,0.907277,0.872172,0.810854,0.739755,0.659590,0.586202,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,0.899701,0.925065,0.953176,0.970737,0.972880,0.987581,0.999396,0.947425,0.817473,0.682324,...,0.715492,0.843744,0.931497,0.928955,0.914313,0.918867,0.894579,0.864759,0.846643,3
488,0.882043,0.903099,0.954086,1.000000,0.982167,0.943872,0.910847,0.907839,0.912979,0.901471,...,0.859483,0.852511,0.853891,0.883268,0.926698,0.945426,0.897190,0.851191,0.831221,3
489,0.792318,0.805000,0.787662,0.767232,0.802106,0.861793,0.868489,0.819355,0.766635,0.748084,...,0.726346,0.774266,0.825273,0.802655,0.740945,0.723790,0.751791,0.762488,0.749903,3
490,0.753196,0.742908,0.727618,0.711634,0.693631,0.685538,0.696426,0.683035,0.657818,0.666619,...,0.607504,0.630874,0.627155,0.624694,0.633411,0.650912,0.673251,0.684370,0.686523,3


In [ ]:
#write the final file to csv
test_final.to_csv('final_1D.csv',header=False,index=False)

In [ ]:
# Now Perform Dimensionality reduction (PCA) on that Dataframe and check
from sklearn.decomposition import PCA

#do PCA and choose componeents as 200
pca = PCA(n_components=200)
x_pca = pca.fit_transform(test_final)
x_pca = pd.DataFrame(x_pca)

# Calculate the variance explained by priciple components
explained_variance = pca.explained_variance_ratio_
print('Variance of each component:', pca.explained_variance_ratio_)
print('\n Total Variance Explained:', round(sum(list(pca.explained_variance_ratio_))*100, 2))

#store the new pca generated dimensions in a dataframe
#store the new pca generated dimensions in a dataframe
pca_df = pd.DataFrame(data = x_pca)
target = pd.Series(result_df.iloc[:,-1], name='target')
final_result_df = pd.concat([pca_df, target], axis=1)
final_result_df

Variance of each component: [1.91717487e-01 1.01387785e-01 8.53225741e-02 6.79764489e-02
 5.24734478e-02 4.27664492e-02 3.25943880e-02 2.79832729e-02
 2.65630346e-02 2.60692080e-02 2.25528591e-02 2.11759364e-02
 1.99382435e-02 1.80724834e-02 1.74501394e-02 1.52291201e-02
 1.41623506e-02 1.30517521e-02 1.21085690e-02 1.13667798e-02
 1.03397016e-02 9.89249006e-03 9.21031219e-03 8.69761412e-03
 8.39417152e-03 7.87003004e-03 7.61666631e-03 7.24386413e-03
 6.91499337e-03 6.24440832e-03 5.92040140e-03 5.64578929e-03
 5.16786168e-03 4.96854836e-03 4.69381551e-03 4.21602951e-03
 3.91918238e-03 3.77566259e-03 3.52327695e-03 3.45177581e-03
 3.24051669e-03 3.09683578e-03 2.92028162e-03 2.72191363e-03
 2.65738384e-03 2.48030550e-03 2.24214201e-03 2.11914455e-03
 2.02050856e-03 1.78443858e-03 1.70565978e-03 1.53836311e-03
 1.46445289e-03 1.46082046e-03 1.32957657e-03 1.20364487e-03
 1.18666656e-03 1.12195900e-03 1.04645772e-03 9.78519649e-04
 8.81963582e-04 8.27320797e-04 7.50145951e-04 7.17785408e

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,target
0,-0.551306,0.247839,-0.437025,-0.115152,1.421921,-0.286450,-1.567482,0.492499,-0.885976,0.563577,...,-0.002865,0.001917,0.002243,0.000178,0.000267,0.001169,0.002930,-0.003499,-0.006889,2
1,-0.880148,-1.010840,1.159707,0.591023,0.205510,-0.459918,-0.774508,-0.352030,1.025419,-0.951890,...,0.000733,-0.006767,-0.003736,-0.001472,-0.003587,-0.000893,-0.002375,0.000780,0.003691,2
2,-0.282394,0.234908,-0.650670,0.633926,-0.888766,-0.421523,1.106065,0.899073,-0.459503,0.060549,...,0.002443,0.001775,0.000384,0.000088,-0.000827,-0.005306,0.004953,0.001445,0.001114,2
3,-1.845713,-0.377831,0.150387,1.383660,-0.216622,0.602231,0.100441,0.427941,0.120217,0.163288,...,-0.004166,-0.003672,-0.002424,-0.003872,-0.001835,-0.003518,-0.002389,0.000500,-0.003781,2
4,-1.139149,-0.360581,0.623470,0.183872,-1.245881,0.539015,0.457348,0.898057,-0.554873,0.153479,...,0.001512,-0.000363,0.000832,-0.002537,0.002772,0.004507,0.000326,0.004496,-0.000131,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,-2.238629,-0.917617,-0.119003,0.185850,-1.655989,0.365520,-0.029819,0.902874,-0.998557,-0.181252,...,0.008345,-0.004454,-0.006524,0.003767,0.000467,-0.000440,0.001684,0.002955,-0.003240,3
488,-1.614420,-1.637329,0.777424,0.437709,-1.593762,0.376330,0.226750,0.022015,-0.830228,0.069965,...,-0.001801,0.003789,0.000271,0.003271,-0.001559,0.003487,-0.001996,0.001514,0.001782,3
489,-0.280526,-0.768810,-0.056903,-1.130444,0.456079,1.638037,-0.483942,0.475226,-0.036395,0.194471,...,-0.000176,-0.004285,0.000499,-0.000427,0.003874,0.000290,0.004767,0.002725,0.003569,3
490,-0.008751,-0.765375,-1.139886,0.757828,0.594430,0.406954,-0.218905,0.618683,0.760060,-0.169538,...,0.002611,0.003897,0.000948,0.003428,0.000394,0.000939,0.001175,-0.001575,-0.003936,3


In [ ]:
#save to dimensionally reduced csv file
final_result_df.to_csv("pca_final.csv")

In [ ]:
import joblib
#save the PCA model
joblib_file='PCA_ECG.pkl'
joblib.dump(pca,joblib_file)

['PCA_ECG.pkl']

In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('knn', KNeighborsClassifier())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)


k_range = list(range(1, 30))
parameters = dict(knn__n_neighbors=k_range)

#input
X = final_result_df.iloc[:,:-1]

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)


cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)

Knn_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
print("Accuracy: {}".format(Knn_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.766497461928934
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        24
           1       0.17      0.05      0.08        20
           2       0.75      0.91      0.82        87
           3       0.87      0.80      0.83        66

    accuracy                           0.77       197
   macro avg       0.63      0.63      0.62       197
weighted avg       0.73      0.77      0.74       197

Tuned Model Parameters: {'knn__n_neighbors': 7}


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline steps:
steps = [('lr', LogisticRegression())]

# Create the pipeline: pipeline
pipeline = Pipeline(steps)

#input
X = final_result_df.iloc[:,:-1]

#target
y=final_result_df.iloc[:,-1]

#parameters for gridsearchcv
c_space = np.logspace(-4, 4, 10)
parameters = {'lr__C': c_space,'lr__penalty': ['l2']}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

#call GridSearchCV and set crossvalscore to 2
cv = GridSearchCV(pipeline,parameters,cv=2)

cv.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred = cv.predict(X_test)
LR_Accuracy = cv.score(X_test, y_test)

In [ ]:
# Compute and print metrics
print("Accuracy: {}".format(LR_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        87
           3       1.00      1.00      1.00        66

    accuracy                           1.00       197
   macro avg       1.00      1.00      1.00       197
weighted avg       1.00      1.00      1.00       197

Tuned Model Parameters: {'lr__C': 1291.5496650148827, 'lr__penalty': 'l2'}


In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Setup the pipeline
steps = [('SVM', SVC())]

pipeline = Pipeline(steps)

#input
X = final_result_df.iloc[:,:-1]

#target
y=final_result_df.iloc[:,-1]


parameters = {'SVM__C':[1, 10, 100],
              'SVM__gamma':[0.1, 0.01]}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=21)

cv = GridSearchCV(pipeline,parameters,cv=3)
cv.fit(X_train,y_train)

y_pred = cv.predict(X_test)
SVM_Accuracy = cv.score(X_test, y_test)

# Compute and print metrics
SVM_Accuracy=cv.score(X_test, y_test)

print("Accuracy: {}".format(SVM_Accuracy))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00        32
           2       1.00      1.00      1.00       109
           3       1.00      1.00      1.00        78

    accuracy                           1.00       246
   macro avg       1.00      1.00      1.00       246
weighted avg       1.00      1.00      1.00       246

Tuned Model Parameters: {'SVM__C': 100, 'SVM__gamma': 0.01}


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier()
model.fit(X_train, y_train)


XGBClassifier(objective='multi:softprob')

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {}".format(accuracy))
print(classification_report(y_test, y_pred))

Accuracy: 0.9024390243902439
              precision    recall  f1-score   support

           0       0.93      0.96      0.95        27
           1       0.91      0.62      0.74        32
           2       0.86      0.95      0.90       109
           3       0.96      0.92      0.94        78

    accuracy                           0.90       246
   macro avg       0.91      0.87      0.88       246
weighted avg       0.91      0.90      0.90       246



In [ ]:
# Import the necessary modules for ML model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
import joblib
#input
X = final_result_df.iloc[:,:-1]
#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.4,random_state=42)

knn =  KNeighborsClassifier(n_neighbors=1)

knn.fit(X_train,y_train)

joblib_file='knn_model_test.pkl'
joblib.dump(knn,joblib_file)

['knn_model_test.pkl']

In [ ]:
# Import the necessary modules for ML model
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

#input
X = pd.read_csv('final_1D.csv',header=None)

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=21)

svm=SVC(C=10,gamma=0.01)

svm.fit(X_train,y_train)

joblib_file='svm_model_test.pkl'
joblib.dump(svm,joblib_file)

['svm_model_test.pkl']

In [ ]:
# Importing required modules
from sklearn import linear_model, tree, ensemble
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
#input
X = final_result_df.iloc[:,0:-1]

#target
y=final_result_df.iloc[:,-1]

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
# Stacking of ML Models
eclf = VotingClassifier(estimators=[
    ('SVM', SVC(probability=True)),
    ('knn', KNeighborsClassifier()),
    ('rf', ensemble.RandomForestClassifier()),
    ('bayes',GaussianNB()),
    ('logistic',LogisticRegression()),
    ], voting='soft')

In [ ]:
# Hyperparameter Tuning using gridSearch
params = {'SVM__C':[1, 10, 100],
          'SVM__gamma':[0.1, 0.01],
          'knn__n_neighbors': [1,5,5],
          'rf__n_estimators':[200, 400],
          }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
voting_clf = grid.fit(X_train, y_train)

print(grid.best_params_)
y_pred = voting_clf.predict(X_test)

# Compute and print metrics
Voting_Accuracy=voting_clf.score(X_test, y_test)

print("Accuracy: {}".format(Voting_Accuracy))
print(classification_report(y_test, y_pred))
print(voting_clf.best_params_)

{'SVM__C': 10, 'SVM__gamma': 0.01, 'knn__n_neighbors': 5, 'rf__n_estimators': 200}
Accuracy: 0.9391891891891891
              precision    recall  f1-score   support

           0       0.82      0.93      0.87        15
           1       0.90      0.53      0.67        17
           2       0.93      1.00      0.96        64
           3       1.00      1.00      1.00        52

    accuracy                           0.94       148
   macro avg       0.91      0.87      0.88       148
weighted avg       0.94      0.94      0.93       148

{'SVM__C': 10, 'SVM__gamma': 0.01, 'knn__n_neighbors': 5, 'rf__n_estimators': 200}


In [ ]:
# open a file, where you ant to store the data
file = open('Heart_Disease_Prediction_using_ECG.pkl', 'wb')
# dump information to that file
pickle.dump(voting_clf, file)